<a href="https://colab.research.google.com/github/lmencisoe/ADL/blob/main/taller%204/Taller4_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from pandas_profiling import ProfileReport
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import ConfusionMatrixDisplay, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
import tensorflow as tf
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
import seaborn as sns
import warnings
import re
warnings.filterwarnings('ignore')

In [13]:
df_1 = pd.read_json('https://raw.githubusercontent.com/lmencisoe/CDA/main/taller%204/DataSet_Entrenamiento_v1.json')
df_2 = pd.read_json('https://raw.githubusercontent.com/lmencisoe/CDA/main/taller%204/DataSet_Entrenamiento_v2.json')

In [14]:
df_1['TotalCharges'] = df_1['TotalCharges'].replace("", 0)
df_1['TotalCharges'] = df_1['TotalCharges'].astype(float)
df_2['TotalCharges'] = df_2['TotalCharges'].replace("", 0)
df_2['TotalCharges'] = df_2['TotalCharges'].astype(float)

In [15]:
X_total = df_1.drop(['Churn', 'customerID'], axis=1)
df_1['target'] = np.where(df_1['Churn']!= 'Yes', 1, 0)
Y_total = df_1['target'].astype(float)
#X_total = X_total.drop(['target'], axis=1)

In [16]:
X_total2 = df_2.drop(['Churn', 'customerID'], axis=1)
df_2['target'] = np.where(df_2['Churn']!= 'Yes', 1, 0)
Y_total2 = df_2['target'].astype(float)
#X_total2 = X_total2.drop(['target'], axis=1)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X_total, Y_total, test_size=0.2, random_state=2022)

In [37]:
cat_features = X_total.select_dtypes(exclude=["float64", "int64"]).columns.to_list()
numeric_features = X_total.select_dtypes(["float64", "int64"]).columns.to_list()

categorical_transformer = Pipeline(
            steps=[
                ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                ('encoder', OneHotEncoder())
            ]
        )    

numeric_transformer = Pipeline(
            steps=[
                ('imputer', SimpleImputer(strategy='most_frequent')),
                ('scaler', StandardScaler())
            ]
        )

preprocessor = ColumnTransformer(
            transformers=[
                ('numerical', numeric_transformer, numeric_features),
                ('categorical', categorical_transformer, cat_features)
            ]
        )

In [27]:
xgb_pipeline = Pipeline(
            [
                ("preprocessor", preprocessor),
                (
                    "classifier", XGBClassifier(),
                ),
            ]
        )

In [35]:
xgb_param_grid = {
    'classifier__learning_rate': np.arange(0.05, 1, 0.05),
    'classifier__max_depth': np.arange(3, 10, 1),
    'classifier__n_estimators': np.arange(50, 200, 50)
}

# Perform RandomizedSearchCV
randomized_roc_auc_xgb = RandomizedSearchCV(estimator=xgb_pipeline, 
                                            param_distributions=gbm_param_grid,
                                            n_iter=50,
                                            scoring='roc_auc',
                                            cv=3, 
                                            verbose=1)

# Fit the estimator
with tf.device('/device:GPU:0'): 
  randomized_roc_auc_xgb.fit(X_train, y_train)

# Compute metrics
print('Score: ', randomized_roc_auc.best_score_)
#print('Estimator: ', randomized_roc_auc.best_estimator_)

Fitting 2 folds for each of 50 candidates, totalling 100 fits
Score:  0.840700712356648
Estimator:  Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['SeniorCitizen', 'tenure',
                                                   'MonthlyCharges',
                                                   'TotalCharges']),
                                                 ('categorical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='